In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

## Dizajn regulatora za sustav masa-opruga-prigušivač

Ovaj primjer prikazuje dizajn regulatora za sustav masa-opruga-prigušivač opisan sljedećim jednadžbama prostora stanja ($m=2,\,k=4\,\text{i}\,c=1$):

\begin{cases}
\dot{\textbf{x}}=\underbrace{\begin{bmatrix}0&1\\-\frac{4}{2}&-\frac{1}{2}\end{bmatrix}}_{A}\textbf{x}+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\end{bmatrix}}_{B}\textbf{u} \\ \\
\textbf{y}=\underbrace{\begin{bmatrix}1&0\end{bmatrix}}_{C}\textbf{x}\,.
\end{cases}

Cilj upravljanja je dobiti maksimalno prekoračenje od 10% i zajamčiti nultu pogrešku u odzivu za referentni step-signal.


Odabrani polovi su $\lambda_{c\,1,2}=-1\pm1.5i$ za upravljivi sustav i $\lambda_{o\,1,2}=-15$ rad/s za promatrača, oba dobivena s obzirom na matrice pojačanja  $K=\begin{bmatrix}\frac{5}{2}&3\end{bmatrix}$ i $L=\begin{bmatrix}\frac{59}{2}&\frac{833}{4}\end{bmatrix}^T$. Rezultirajuća struktura kontrolera je:

<img src="Images\Block_regulator2.png">

i, da bi se postigla nulta pogreška odziva, referentni signal se dijeli statičkim pojačanjem prijenosne funkcije zatvorene petlje od referentne vrijednosti do $y$.

Sustav je simuliran u nastavku.

### Kako koristiti ovaj interaktivni primjer?

- Izmijenite početne vrijednosti stanja promatrača i svojstvene vrijednosti promatrača te promatrajte odziv sustava.
- Pokušajte postići savršeno praćenje sinusoidalnog referentnog signala s periodom od 3 s.

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix('0 1;-2 -0.5')
B = numpy.matrix('0; 0.5')
C = numpy.matrix('1 0')
X0 = numpy.matrix('0.5; 0.5')
K = numpy.matrix([5/2,3.])
L = numpy.matrix([[59/2],[833/4]])

Aw = matrixWidget(2,2)
Aw.setM(A)
Bw = matrixWidget(2,1)
Bw.setM(B)
Cw = matrixWidget(1,2)
Cw.setM(C)
X0w = matrixWidget(2,1)
X0w.setM(X0)
Kw = matrixWidget(1,2)
Kw.setM(K)
Lw = matrixWidget(2,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig1c.setM(numpy.matrix([-1.])) 
eig2c.setM(numpy.matrix([[-1.],[-1.5]]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig1o.setM(numpy.matrix([-15.])) 
eig2o.setM(numpy.matrix([[-15.],[0.]]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Postavi K i L', 'Postavi svojstvene vrijednosti'],
    value= 'Postavi svojstvene vrijednosti',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 kompleksnih svojstvenih vrijednosti', '2 kompleksne svojstvene vrijednosti'],
    value= '2 kompleksne svojstvene vrijednosti',
    description='Kompleksne svojstvene vrijednosti:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impuls', 'step', 'sinus', 'Pravokutni val'],
    value='step',
    description='Tip referentnog signala:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description='Referentni signal:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.01,
    max=4,
    step=0.01,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

gain_id_w = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

m = widgets.FloatSlider(
    value=2,
    min=0.1,
    max=10.0,
    step=0.1,
    description='m [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
k = widgets.FloatSlider(
    value=4,
    min=0,
    max=10.0,
    step=0.1,
    description='k [N/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
c = widgets.FloatSlider(
    value=1,
    min=0,
    max=10.0,
    step=0.1,
    description='c [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

time = widgets.FloatText(
    value=5.,
    description='Time',
    disabled=False
)

In [5]:
# Support functions

def eigen_choice(sele):
    if sele == '0 kompleksnih svojstvenih vrijednosti':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 kompleksne svojstvene vrijednosti':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'Postavi K i L':
        method = 1
        sele.disabled = True
    if selm == 'Postavi svojstvene vrijednosti':
        method = 2
        sele.disabled = False
    return method

In [6]:
sols = numpy.linalg.eig(A)

def main_callback(m, k, c, X0w, K, L, eig1c, eig2c, eig1o, eig2o, u, period, selm, sele, selu, time, DW):
    A = numpy.matrix([[0,1],[-k/m,-c/m]])
    B = numpy.matrix([[0],[1/m]])
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B, [numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    Gs = sss(A,B,numpy.vstack((C,[0,0])),[[0],[1]])
    Os = sss(A-L*C,numpy.hstack((L,B)),-K,[0,0])
    Gas = control.append(Gs,Os)
    sys = control.connect(Gas,[[2,1],[3,2],[1,3]],[1],[1])
    
    Gs_id = sss(A,B,sym.eye(2),sym.zeros(2,1))
    Fs_id = control.series(K,Gs_id)
    A1 = numpy.matrix(Fs_id.A-Fs_id.B*Fs_id.C)
    B1 = numpy.matrix(Fs_id.B*sym.Matrix([[1],[0]]))
    C1 = numpy.matrix(sym.Matrix([1,0]).T*Fs_id.C)
    sys_id = sss(A1,B1,C1,0)

    
    dcgain = control.dcgain(sys)
    t = numpy.linspace(0, 1000, 2)
    t, y = control.step_response(sys_id,t)
    dcgain_id = y[-1]
    gain_w.value = dcgain
    gain_id_w.value = dcgain_id
    if dcgain != 0 and dcgain_id != 0:
        u1 = u/gain_w.value
        u2 = u/gain_id_w.value
    else:
        print('Inverzno pojačanje postavljeno na 0 i promijenjeno na 1')
        u1 = u/1
        u2 = u/1
    
    solc = numpy.linalg.eig(sys.A)
    solo = numpy.linalg.eig(A-L*C-B*K)
    print('Svojstvene vrijednosti sustava su:', round(sols[0][0],2),'i', round(sols[0][1],2))
    print('Svojstvene vrijednosti upravljivog sustava u zatvorenoj petlji su:', 
          round(solc[0][0],2),',', 
          round(solc[0][1],2),',', 
          round(solc[0][2],2),'i',
          round(solc[0][3],2))
    print('')
    print('Statičko pojačanje sustava zatvorene petlje(od referentnog ulaza do izlaza) je: %.5f' %dcgain)
    print('Statičko pojačanje idealnog sustava zatvorene petlje(od referentnog ulaza do izlaza) je: %.5f' %dcgain_id)
    
    X0w1 = numpy.matrix([[0],[0],[X0w[0,0]],[X0w[1,0]]])
    if time != 0:
        T = numpy.linspace(0, time, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)

    if selu == 'impuls': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        U2 = [0 for t in range(0,len(T))]
        U2[0] = u2
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0])
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        U2 = [u2 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0])
    if selu == 'sinus':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        U2 = u2*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0])
    if selu == 'Pravokutni val':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U2 = u2*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U1,X0w1)
        T, yout_id, xout_id = control.forced_response(sys_id,T,U2,[0, 0])
    
    fig = plt.figure(num='Simulacija', figsize=(14,12))
    mag, phase, omega = control.bode_plot(sys,Plot = False)
    mag = control.mag2db(mag)
    phase = phase*180/numpy.pi
    fig.add_subplot(221)
    plt.title('Bodeov graf: amplituda')
    plt.semilogx(omega,mag)
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('A [dB]')
    plt.grid(True,which="both")
    
    fig.add_subplot(223)
    plt.title('Bodeov graf: faza')
    plt.semilogx(omega,phase)
    plt.xlabel('$\omega$ [rad/s]')
    plt.ylabel('Faza [deg]')
    plt.grid(True,which="both")
    
    fig.add_subplot(222)
    plt.title('Izlazni odziv')
    plt.ylabel('Izlaz ($x_1$)')
    plt.plot(T,xout[2],T,xout[0],T,xout_id[0],'g',T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{1estimirani}$','$y = x_{1realni}$','$x_{1idealni}$','referentni signal'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Odziv drugog stanja')
    plt.ylabel('$x_2$')
    plt.plot(T,xout[3],T,xout[1],T,xout_id[1],'g')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_{2estimirani}$','$x_{2realni}$','$x_{2idealni}$'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()

   
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1c, 
                                          eig2c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 est.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Svojstvene vrijednosti:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          widgets.Label(' ',border=3),
                                          widgets.VBox([widgets.Label('Inverzno referentno pojačanje:',border=3),
                                                        widgets.Label('Inverzno idealno referentno pojačanje:',border=3)]),
                                          widgets.VBox([gain_w,gain_id_w])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([m, 
                                          k, 
                                          c]),
                            widgets.HBox([u, 
                                          period, 
                                          time]),
                            START])
out = widgets.interactive_output(main_callback, {'m':m, 'k':k, 'c':c, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig1o':eig1o, 'eig2o':eig2o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 
                                                 'time':time, 'DW':DW})
out.layout.height = '900px'
display(out, alltogether)

Output(layout=Layout(height='900px'))